In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/jessenb16/CIFAR_10_Classification.git
%cd CIFAR_10_Classification

In [ ]:
# Define file paths in /kaggle/working
csv_file = '/kaggle/working/saved_data/resnet1_metrics.csv'
#current_checkpoint_file = '/kaggle/input/checkpoint/ckpt_resnetv1_e200.pth'
checkpoint_dir = '/kaggle/working/saved_models'

# Create directories if they don't exist
os.makedirs(os.path.dirname(csv_file), exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_file = os.path.join(checkpoint_dir, 'ckpt.pth')

In [ ]:
import torch.optim as optim
import torchvision.transforms as transforms
from src.model import create_model, count_parameters
from src.train import main as train
from src.evaluate import main as evaluate

#Choose model parameters
BLOCKS_PER_LAYER = [3,3,4]
CHANNELS_PER_LAYER = [60,120,240]
KERNEL_SIZE = 3
SKIP_KERNEL_SIZE = 1
POOL_SIZE = 8

EPOCHS = 50

TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128
AUGMENTATIONS = [
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5),
]
NUM_WORKERS = 4

RESUME = True

In [ ]:
#Create Model
try:
    model=create_model(
        blocks_per_layer = BLOCKS_PER_LAYER,
        channels_per_layer = CHANNELS_PER_LAYER,
        kernel_size = KERNEL_SIZE,
        skip_kernel_size = SKIP_KERNEL_SIZE,
        pool_size = POOL_SIZE
    )
    print("Number of parameters =",count_parameters(model))
except AssertionError as e:
    print(f'Failed to create model: {e}')
    exit(1)



In [ ]:
#Train model
try:
    LEARNING_RATE = 0.001
    SMOOTHING = 0.05
        
    # Define optimizer
    OPTIMIZER = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4, nesterov=True)
    # OPTIMIZER = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
        
    # Define scheduler
    # SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER, step_size=10, gamma=0.1)
    # SCHEDULER = optim.lr_scheduler.MultiStepLR(OPTIMIZER, milestones=[30, 60, 90], gamma=0.1)
    SCHEDULER = optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=EPOCHS)
        
    # If no scheduler, set to None
    SCHEDULER = SCHEDULER if 'SCHEDULER' in locals() and SCHEDULER is not None else None

    # train(model, EPOCHS)
    train(model, EPOCHS, train_batch_size=TRAIN_BATCH_SIZE, test_batch_size=TEST_BATCH_SIZE, augmentations=AUGMENTATIONS,
            optimizer=OPTIMIZER, scheduler=SCHEDULER, learning_rate=LEARNING_RATE, smoothing=SMOOTHING, resume=RESUME)
except (ValueError, TypeError) as e:
    print(f'Training failed: {e}')
    exit(1)

In [ ]:
#Evaluate hidden dataset
try:
    evaluate()
except FileNotFoundError as e:
    print(f'Evaluation failed: {e}')